In [1]:
import os
import copy
import numpy as np
import pandas as pd
import scanpy as sc

from scipy import sparse

In [2]:
def maskPerCol(data_train, masked_prob, seed=3407):
    """
    https://github.com/Lav-i/GNNImpute/blob/main/data/mask.py
    将表达矩阵中每列非零的值随机置为0并返回，同时返回置为0的元素的坐标
    :param data_train: 表达矩阵
    :param masked_prob: 置0比例
    :return:
    """
    np.random.seed(seed)
    X_train = copy.deepcopy(data_train)
    rows = []
    cols = []
    for col in range(data_train.shape[1]):
        index_pair_train = np.where(data_train[:, col])
        if index_pair_train[0].shape[0] <= 3:
            continue
        masking_idx_train = np.random.choice(index_pair_train[0].shape[0],
                                             int(index_pair_train[0].shape[0] * masked_prob),
                                             replace=False)
        X_train[index_pair_train[0][masking_idx_train], [col] * masking_idx_train.shape[0]] = 0
        for i in index_pair_train[0][masking_idx_train]:
            rows.append(i)
            cols.append(col)

    return X_train, rows, cols

In [3]:
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--masked_prob', default=0.2, type=float)
parser.add_argument('--dataset', default='Zeisel', type=str)
parser.add_argument('--downsample', default=1.0, type=float)

_StoreAction(option_strings=['--downsample'], dest='downsample', nargs=None, const=None, default=1.0, type=<class 'float'>, choices=None, required=False, help=None, metavar=None)

In [4]:
args, unknown = parser.parse_known_args()

In [5]:
adata = sc.read_h5ad('D:/MyData/scEDGE_Generation/data/%s/processed/%s.h5ad' % (args.dataset, args.dataset))
sc.pp.normalize_total(adata)
adata.raw = adata

D:\Anaconda3\envs\PyTorch\lib\site-packages\anndata\_core\anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [6]:
path = 'D:/MyData/scEDGE_Generation/data/%s/masked' % args.dataset

if not os.path.exists(path):
    os.makedirs(path)

masked, masking_row, masking_col = maskPerCol(adata.raw.X.A, args.masked_prob)

In [7]:
pd.DataFrame(masked, index=adata.obs.index, columns=adata.var.index) \
    .T.to_csv(path + '/%s_%s.csv' % (args.dataset, str(args.masked_prob).replace('.', '')))

In [8]:
adata.X = sparse.csr_matrix(masked)
adata.write(path + '/%s_%s.h5ad' % (args.dataset, str(args.masked_prob).replace('.', '')))

In [9]:
maskIndex = sparse.coo_matrix(([1] * len(masking_col), (masking_row, masking_col)))

In [10]:
sparse.save_npz(path + '/%s_maskIndex_%s.csv' % (args.dataset, str(args.masked_prob).replace('.', '')), maskIndex)